## Begin

### Import Statements

In [1]:
import pandas as pd
import itertools
import json
import numpy as np
import re

### Load DataFrame

#### Constants

In [2]:
algorithms = ['MDS', 'TSNE', 'RANDOM']

#### Function to Load

In [3]:
def ret_df(filepath: str):
    # Read the JSON file
    with open(filepath, 'r') as f:
        data = json.load(f)

    # Create a list to store the data
    rows = []
    for dataset_key, values in data.items():
        # Split dataset name and number
        if 'fashion_mnist' in dataset_key:
            dataset_key = dataset_key.replace('fashion_mnist', 'fashion MNIST')

        dataset, run = dataset_key.split('_')
        
        # For each metric in the values
        for metric_key, value in values.items():
            # Split algorithm and metric type
            algorithm, metric = metric_key.split('_')
            
            # Add to rows
            rows.append({
                'Dataset': dataset,
                'Run': int(run),
                'Algorithm': algorithm,
                'Metric': metric,
                'Value': value
            })

        df = pd.DataFrame(rows)
    # Pivot the DataFrame to get the desired structure
    df = df.pivot_table(
        index=['Dataset', 'Run'],
        columns=['Metric', 'Algorithm'],
        values='Value'
    )

    # Get rid of the 'UMAP' column
    df = df.drop(columns=['UMAP'], level=1)

    # Rename the metric level of the columns
    metric_mapping = {
        'raw': 'RS',
        'norm': 'NS', 
        'sheppard': 'SGS',
        'scalenorm': 'SNS',
        'kruskal': 'NMS',
    }
    df.columns = pd.MultiIndex.from_tuples(
        [(metric_mapping[m], a) for m, a in df.columns],
        names=['Metric', 'Algorithm']
    )

    df = df[['RS', 'NS', 'SGS', 'NMS', 'SNS']]
    
    # Normalize SGS to between 0 and 1, lower is better
    df.loc[:, ('SGS', algorithms)] = (1 - df.loc[:, ('SGS', algorithms)]) / 2

    return df

#### Create DataFrames for out1x and out10x

In [4]:
stress_1x_df = ret_df('out1x.json')
stress_10x_df = ret_df('out10x.json')

In [5]:
stress_df = pd.read_csv('csv_files/stress_metrics.csv', index_col=[0, 1, 2])
algorithms = ['MDS', 'TSNE', 'RANDOM']
stress_df = stress_df.loc[:, :, algorithms]

### When is MDS much better than t-SNE according to SNS, and when are they equally performant?

In [6]:
# When is SNS score for MDS 2.5 smaller than that for t-SNE?
sns_scores = stress_1x_df['SNS']
sns_scores[sns_scores['MDS'] * 2.5 < sns_scores['TSNE']]

Algorithm           MDS    RANDOM      TSNE
Dataset   Run                              
epileptic 0    0.194820  0.740852  0.663310
          1    0.194682  0.739204  0.647710
          2    0.194647  0.737914  0.663607
          3    0.194777  0.735219  0.658632
          4    0.194737  0.735357  0.663111
          5    0.194833  0.738393  0.659583
          6    0.194770  0.740223  0.657779
          7    0.194811  0.737690  0.658808
          8    0.195043  0.737385  0.661685
          9    0.195034  0.736797  0.658191
iris      0    0.101929  0.653551  0.284364
          2    0.102702  0.652854  0.259032
          3    0.102206  0.653824  0.260475
          4    0.102947  0.655940  0.261980
          5    0.102306  0.641000  0.265888
          6    0.101547  0.652494  0.254921
          7    0.101935  0.645637  0.263326
          8    0.101721  0.640964  0.260975

In [7]:
# When is the SNS score for t-SNE not more than 20% larger that that of MDS
sns_scores[sns_scores['MDS'] * 1.2 > sns_scores['TSNE']]

Algorithm            MDS    RANDOM      TSNE
Dataset    Run                              
bank       3    0.355007  0.447310  0.423199
coil20     1    0.307880  0.467834  0.363263
fmd        3    0.334249  0.458207  0.399008
           5    0.334264  0.455819  0.399184
hatespeech 0    0.392788  0.445713  0.424685
...                  ...       ...       ...
wine       5    0.589809  0.699367  0.596236
           6    0.585654  0.689790  0.584413
           7    0.598024  0.707554  0.583818
           8    0.596811  0.706253  0.588388
           9    0.578324  0.698186  0.590791

[73 rows x 3 columns]

In [8]:
pct_score_diff = ((sns_scores['TSNE'] - sns_scores['MDS'])/sns_scores['MDS']) * 100
print(f"Min : {pct_score_diff.min():.2f}")
print(f"Max : {pct_score_diff.max():.2f}")
print(f"Mean : {pct_score_diff.mean():.2f}")
print(f"Standard deviation : {pct_score_diff.std():.2f}")
print(f"Median : {pct_score_diff.median():.2f}")

Min : -5.34
Max : 240.93
Mean : 47.57
Standard deviation : 53.46
Median : 26.10


## Percent Orders

In [9]:
def get_pct_order_df(df):
    permutations = itertools.permutations(algorithms)

    results = {}

    for perm in permutations:
        condition = pd.DataFrame(True, index=df.index, columns=df.columns.get_level_values(0).unique())
        for i in range(1, len(perm)):
            condition &= df.xs(perm[i-1], axis=1, level=1) <= df.xs(perm[i], axis=1, level=1)

        percentage = (condition).mean() * 100
        # results[" < ".join(perm)] = f"{percentage:.3}%"
        results[" < ".join(perm)] = percentage

    pct_order_df = pd.DataFrame(results).T
    # Round the following styler to 2nd decimal place
    return pct_order_df.style.background_gradient('Blues', vmin=0, vmax=150).format(lambda x : f"{x:.1f}%")

In [10]:
get_pct_order_df(stress_1x_df)

Metric,RS,NS,SGS,NMS,SNS
MDS < TSNE < RANDOM,4.2%,4.2%,93.3%,81.7%,90.0%
MDS < RANDOM < TSNE,83.3%,83.3%,0.0%,18.3%,8.3%
TSNE < MDS < RANDOM,12.5%,12.5%,6.7%,0.0%,1.7%
TSNE < RANDOM < MDS,0.0%,0.0%,0.0%,0.0%,0.0%
RANDOM < MDS < TSNE,0.0%,0.0%,0.0%,0.0%,0.0%
RANDOM < TSNE < MDS,0.0%,0.0%,0.0%,0.0%,0.0%


In [11]:
get_pct_order_df(stress_10x_df)

Metric,RS,NS,SGS,NMS,SNS
MDS < TSNE < RANDOM,0.0%,0.0%,93.3%,81.7%,90.0%
MDS < RANDOM < TSNE,7.5%,7.5%,0.0%,18.3%,8.3%
TSNE < MDS < RANDOM,12.5%,12.5%,6.7%,0.0%,1.7%
TSNE < RANDOM < MDS,0.0%,0.0%,0.0%,0.0%,0.0%
RANDOM < MDS < TSNE,80.0%,80.0%,0.0%,0.0%,0.0%
RANDOM < TSNE < MDS,0.0%,0.0%,0.0%,0.0%,0.0%


## Correlation

In [12]:
score_corr_df = stress_1x_df.stack('Algorithm', future_stack=True).corr('spearman')
score_corr_df

Metric,RS,NS,SGS,NMS,SNS
Metric,,,,,
RS,1.000000,0.775578,0.091234,0.213297,0.296248
NS,0.775578,1.000000,0.160423,0.259010,0.322712
SGS,0.091234,0.160423,1.000000,0.888302,0.845332
NMS,0.213297,0.259010,0.888302,1.000000,0.879197
SNS,0.296248,0.322712,0.845332,0.879197,1.000000


In [13]:
def corr_styler(df: pd.DataFrame):
    df = df
    df = df.round(2)
    mask = np.tril(np.ones(df.shape), k=-1).astype(bool)
    upper_triangled = df.mask(mask, -2)
    return upper_triangled.style.background_gradient('Blues', vmin=0, vmax=2).format(lambda x : '-' if x == -2 else x)
    
corr_styler(score_corr_df)

Metric,RS,NS,SGS,NMS,SNS
Metric,,,,,
RS,1.0,0.78,0.09,0.21,0.3
NS,-,1.0,0.16,0.26,0.32
SGS,-,-,1.0,0.89,0.85
NMS,-,-,-,1.0,0.88
SNS,-,-,-,-,1.0


## Latex

In [42]:
def get_latex_code(styler: pd.io.formats.style.Styler, entry_type='corr'):
    """
    Parameters
    ----------
    styler : 
    entry_type : 'corr' or 'pct'
    """
    corr_df = styler.data
    entries = corr_df.to_numpy()
    index = corr_df.index.tolist()
    cols = corr_df.columns.tolist()

    corr_css = styler.to_html()
    # print(corr_css)
    n_cols = len(styler.columns)
    n_rows = len(styler.index)
    colors = [["F"*6] * n_cols for _ in range(n_rows)]

    
    matches = re.findall(r'#T_\w+_row(\d+)_col(\d+)[^}]+background-color: #(\w+);', corr_css)

    # Fill the grid with the extracted background colors
    for match in matches:
        # print(match)
        row, col, color = int(match[0]), int(match[1]), match[2]
        colors[row][col] = color.upper()
    colors = np.array(colors)

    latex_code = f"""\\begin{{table}}[ht]
\\centering
\\caption{{}}
\\begin{{tabularx}}{{\\linewidth}}{{|p{{2cm}}| {'X ' * n_cols} |}}
\hline
"""

    for col in cols:
        latex_code += f"& \\small{{{col}}} "
    latex_code += "\\\\ \\hline \n"

    if entry_type == 'pct':
        index = [idx.replace(" < ", "$<$").replace("TSNE", "t-SNE").replace("RANDOM", "RND") for idx in index]

    for i in range(n_rows):
        latex_code += index[i]
        for color, entry in zip(colors[i], entries[i]):
            if entry_type == 'corr':
                tab_entry = entry if entry != -2 else '--'
            elif entry_type == 'pct':
                tab_entry = f'{float(entry):.1f}\\%'
            else:
                raise ValueError(f"entry_type should be either 'corr' or 'pct'. Got {entry_type} instead")
            latex_code += f" & \\cellcolor[HTML]{{{color}}} {tab_entry} "
        latex_code += "\\\\ \\hline \n"
    
    latex_code += """\\end{tabularx}
\\newline
\\label{tab:}
\\end{table}"""

    print(latex_code)
    return colors


In [43]:

get_latex_code(corr_styler(score_corr_df), entry_type='corr')

\begin{table}[ht]
\centering
\caption{}
\begin{tabularx}{\linewidth}{|p{2cm}| X X X X X  |}
\hline
& \small{RS} & \small{NS} & \small{SGS} & \small{NMS} & \small{SNS} \\ \hline 
RS & \cellcolor[HTML]{6AAED6} 1.0  & \cellcolor[HTML]{99C7E0} 0.78  & \cellcolor[HTML]{EEF5FC} 0.09  & \cellcolor[HTML]{E3EEF8} 0.21  & \cellcolor[HTML]{D9E8F5} 0.3 \\ \hline 
NS & \cellcolor[HTML]{F7FBFF} --  & \cellcolor[HTML]{FFFFFF} 1.0  & \cellcolor[HTML]{E7F1FA} 0.16  & \cellcolor[HTML]{DDEAF7} 0.26  & \cellcolor[HTML]{D8E7F5} 0.32 \\ \hline 
SGS & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} 1.0  & \cellcolor[HTML]{82BBDB} 0.89  & \cellcolor[HTML]{8ABFDD} 0.85 \\ \hline 
NMS & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} 1.0  & \cellcolor[HTML]{84BCDB} 0.88 \\ \hline 
SNS & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} --  & \cellcolor[HTML]{FFFFFF} -

array([['6AAED6', '99C7E0', 'EEF5FC', 'E3EEF8', 'D9E8F5'],
       ['F7FBFF', 'FFFFFF', 'E7F1FA', 'DDEAF7', 'D8E7F5'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', '82BBDB', '8ABFDD'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', '84BCDB'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF']], dtype='<U6')

In [44]:
get_latex_code(get_pct_order_df(stress_1x_df), entry_type='pct')

\begin{table}[ht]
\centering
\caption{}
\begin{tabularx}{\linewidth}{|p{2cm}| X X X X X  |}
\hline
& \small{RS} & \small{NS} & \small{SGS} & \small{NMS} & \small{SNS} \\ \hline 
MDS$<$t-SNE$<$RND & \cellcolor[HTML]{F2F7FD} 4.2\%  & \cellcolor[HTML]{FFFFFF} 4.2\%  & \cellcolor[HTML]{4292C6} 93.3\%  & \cellcolor[HTML]{5CA4D0} 81.7\%  & \cellcolor[HTML]{4A98C9} 90.0\% \\ \hline 
MDS$<$RND$<$t-SNE & \cellcolor[HTML]{58A1CF} 83.3\%  & \cellcolor[HTML]{FFFFFF} 83.3\%  & \cellcolor[HTML]{F7FBFF} 0.0\%  & \cellcolor[HTML]{DFEBF7} 18.3\%  & \cellcolor[HTML]{ECF4FB} 8.3\% \\ \hline 
t-SNE$<$MDS$<$RND & \cellcolor[HTML]{E7F0FA} 12.5\%  & \cellcolor[HTML]{FFFFFF} 12.5\%  & \cellcolor[HTML]{EEF5FC} 6.7\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{F5FAFE} 1.7\% \\ \hline 
t-SNE$<$RND$<$MDS & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\% \\ \hline 
RND$<$MDS$<$t-SNE & \c

array([['F2F7FD', 'FFFFFF', '4292C6', '5CA4D0', '4A98C9'],
       ['58A1CF', 'FFFFFF', 'F7FBFF', 'DFEBF7', 'ECF4FB'],
       ['E7F0FA', 'FFFFFF', 'EEF5FC', 'FFFFFF', 'F5FAFE'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF']], dtype='<U6')

In [45]:
get_latex_code(get_pct_order_df(stress_10x_df), entry_type='pct')

\begin{table}[ht]
\centering
\caption{}
\begin{tabularx}{\linewidth}{|p{2cm}| X X X X X  |}
\hline
& \small{RS} & \small{NS} & \small{SGS} & \small{NMS} & \small{SNS} \\ \hline 
MDS$<$t-SNE$<$RND & \cellcolor[HTML]{F7FBFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{4292C6} 93.3\%  & \cellcolor[HTML]{5CA4D0} 81.7\%  & \cellcolor[HTML]{4A98C9} 90.0\% \\ \hline 
MDS$<$RND$<$t-SNE & \cellcolor[HTML]{EEF5FC} 7.5\%  & \cellcolor[HTML]{FFFFFF} 7.5\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{DFEBF7} 18.3\%  & \cellcolor[HTML]{ECF4FB} 8.3\% \\ \hline 
t-SNE$<$MDS$<$RND & \cellcolor[HTML]{E7F0FA} 12.5\%  & \cellcolor[HTML]{FFFFFF} 12.5\%  & \cellcolor[HTML]{FFFFFF} 6.7\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{F5FAFE} 1.7\% \\ \hline 
t-SNE$<$RND$<$MDS & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\%  & \cellcolor[HTML]{FFFFFF} 0.0\% \\ \hline 
RND$<$MDS$<$t-SNE & \cel

array([['F7FBFF', 'FFFFFF', '4292C6', '5CA4D0', '4A98C9'],
       ['EEF5FC', 'FFFFFF', 'FFFFFF', 'DFEBF7', 'ECF4FB'],
       ['E7F0FA', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'F5FAFE'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF'],
       ['60A7D2', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF'],
       ['FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF', 'FFFFFF']], dtype='<U6')

## Q3 NS vs. SNS comparison

### Load data

In [14]:
# Download projections used in Espadoto Paper
import os
import subprocess

url = "https://mespadoto.github.io/proj-quant-eval/data/projections/projections.tar.gz"
tar_file = "projections.tar.gz"
extract_dir = "projections"

# Create directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir, exist_ok=True)

    subprocess.run(["wget", url], check=True) # Download tar file
    subprocess.run(["tar", "-xzf", tar_file, "-C", extract_dir], check=True) # Extract contents
    os.remove(tar_file) # Delete tar file

In [ ]:
!wget https://mespadoto.github.io/proj-quant-eval/data/projections/projections.tar.gz && mkdir -p projections && tar -xzf projections.tar.gz -C projections && rm projections.tar.gz



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://mespadoto.github.io/pr]87[Files: 0  Bytes: 0  [0 B/s] Re]87Saving 'projections.tar.gz'
87projections.tar.gz    15% [===>                          ]    4.42M    --.-KB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87[Files: 0  Bytes: 0  [0 B/s] Re]87projections.tar.gz    35% [=========>                    ]    9.78M    5.35MB/s87projections.tar.gz    52% [==============>               ]   14.71M    5.14MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87[Files: 0  Bytes: 0  [0 B/s] Re]87projections.tar.gz    73% [=====================>        ]   20.58M    5.38MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87projections.tar.gz    93% [==========================>   ]   25.98M    5.38MB/s87projections.tar.gz   100% [=============================>]   27.84M    5.38MB/s87HTTP response 200  [https://mespadoto.github.io/proj-quant-eval/data/projections/projections.tar.gz]
87projections.tar.gz   100% [=============================>]   27.84M    5.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from metrics import Metrics
import tqdm
import os

In [16]:
small_data = {"auto-mpg.npy", "iris.npy", "penguins.npy", "s-curve.npy", "swissroll.npy", "wine.npy"}
# ORL removed because error in Espadoto repo
# Spambase removed because embeddings are missing in Espadoto repo
error_data = {"spambase.npy", "orl.npy"}

results = dict()
results_scale = dict()
datasets = [dataFname for dataFname in os.listdir('datasets') if dataFname not in small_data | error_data]


for dataFname in tqdm.tqdm(datasets):
    X = np.load(f'datasets/{dataFname}')
    dataname = dataFname.split(".")[0]
    results[dataname] = dict()
    results_scale[dataname] = dict()
    for alg in ["T-SNE", "MDS", "UMAP", "LLE", "ISO"]:

        Y = np.loadtxt(f'projections/proj_X_{dataname}_{alg}.csv', delimiter=',')

        M = Metrics(X,Y, setbatch=False)
        results[dataname][alg] = M.compute_normalized_stress()
        results_scale[dataname][alg] = M.compute_scale_normalized_stress()

100%|██████████| 16/16 [01:13<00:00,  4.56s/it]


In [17]:
NS_ranking = pd.DataFrame.from_dict(results, orient='index')
NS_ranking = NS_ranking.rank(axis=1, method='min').astype(int)
SNS_ranking = pd.DataFrame.from_dict(results_scale, orient='index')
SNS_ranking = SNS_ranking.rank(axis=1, method='min').astype(int)

In [18]:
df = pd.concat({"NS": NS_ranking, "SNS": SNS_ranking}, axis=1)
datasets = df.index.unique()
df.columns.names = ["Metric", "Algorithm"]
df

Metric           NS                    SNS                 
Algorithm     T-SNE MDS UMAP LLE ISO T-SNE MDS UMAP LLE ISO
bank              5   1    4   3   2     2   1    4   5   3
cifar10           5   1    3   4   2     3   1    4   5   2
cnae9             5   1    4   3   2     2   1    3   5   4
coil20            5   1    3   2   4     2   1    3   4   5
fashion_mnist     5   1    2   3   4     3   1    4   5   2
fmd               3   1    5   4   2     2   1    4   5   3
har               5   1    3   2   4     2   1    4   5   3
hatespeech        5   1    4   2   3     2   1    5   4   3
hiva              5   1    2   4   3     2   1    3   5   4
imdb              2   1    5   4   3     2   1    3   4   5
secom             5   1    3   2   4     2   1    3   5   4
seismic           5   1    4   2   3     2   1    3   5   4
sentiment         5   1    4   2   3     2   1    3   5   4
sms               4   1    5   3   2     2   1    3   5   4
svhn              5   1    2   4   3     3   1    5   4   2
epileptic         4   1    5   3   2     3   1    5   4   2

#### Statistics

In [19]:
iso2nd = (df['NS', 'ISO'] == 2).mean()
print(f"ISO was 2nd in NS {iso2nd * 100:.2f}% of the time")
lle2nd = (df['NS', 'LLE'] == 2).mean()
print(f"LLE was 2nd in NS {lle2nd * 100:.2f}% of the time")
tsne2nd = (df['SNS', 'T-SNE'] == 2).mean()
print(f"t-SNE was 2nd in SNS {tsne2nd * 100:.2f}% of the time")

ISO was 2nd in NS 37.50% of the time
LLE was 2nd in NS 37.50% of the time
t-SNE was 2nd in SNS 75.00% of the time


In [20]:
tsne_last = (df['NS', 'T-SNE'] == 5).mean()
print(f"t-SNE was last in NS {tsne_last * 100:.2f}% of the time")
lle_last = (df['SNS', 'LLE'] == 5).mean()
print(f"LLE was last in SNS {lle_last * 100:.2f}% of the time")


t-SNE was last in NS 75.00% of the time
LLE was last in SNS 68.75% of the time


##### **SNS**

In [21]:
df['SNS'].mean(axis=0)

Algorithm
T-SNE    2.2500
MDS      1.0000
UMAP     3.6875
LLE      4.6875
ISO      3.3750
dtype: float64

In [ ]:
# Number of times each algorithm got the given ranking under SNS
pd.concat({i: (df['SNS'] == i).sum() for i in range(1, 6)}, axis=1)
    

,1,2,3,4,5
Algorithm,,,,,
T-SNE,0,12,4,0,0
MDS,16,0,0,0,0
UMAP,0,0,8,5,3
LLE,0,0,0,5,11
ISO,0,4,4,6,2


##### **NS**

In [23]:
df['NS'].mean(axis=0)

Algorithm
T-SNE    4.5625
MDS      1.0000
UMAP     3.6250
LLE      2.9375
ISO      2.8750
dtype: float64

In [24]:
# Number of times each algorithm got the given ranking under NS
pd.concat({i: (df['NS'] == i).sum() for i in range(1, 6)}, axis=1)

,1,2,3,4,5
Algorithm,,,,,
T-SNE,0,1,1,2,12
MDS,16,0,0,0,0
UMAP,0,3,4,5,4
LLE,0,6,5,5,0
ISO,0,6,6,4,0


### Visualizations

In [11]:
colors = {
    "MDS": 'darkred',
    "ISO": 'orange',
    "LLE": 'deeppink',
    "UMAP": 'purple',
    "T-SNE": 'darkblue'
}

In [12]:
import matplotlib.colors as mcolors

algs = ['MDS', 'T-SNE', 'UMAP', 'ISO', 'LLE']

# Hack to make it darker
step = 3
cmap = plt.get_cmap('Oranges', len(algs) + step)
hex_colors = [mcolors.to_hex(cmap(i + step)) for i in range(len(algs))]

colors = dict(zip(algs, hex_colors))

In [36]:
n_rows = 2
n_columns = 8
size = 10
fig, axes = plt.subplots(n_rows, n_columns, figsize=(n_columns * 2 * size, n_rows * 5 * size), 
                        #  sharex=True, sharey=True
)
for i, dataset in enumerate(datasets):
    ax = axes.flatten()[i]
    coords = np.column_stack((np.array([0] * 5 + [1] * 5), -df.loc[dataset].to_numpy()))

    for alg, (x,y) in zip(np.tile(df.columns.get_level_values(1).unique(), 2), coords):
        rect = plt.Rectangle((x, y), 1, 1, facecolor=colors[alg], edgecolor='black')
        ax.add_patch(rect)
        ax.text(x + 0.5, y + 0.5, alg, color='white', ha='center', va='center', fontweight='bold', fontsize=150)

    ax.set_xlim(0, 2)
    ax.set_ylim(-5, 0)
    ax.set_xticks([0.5, 1.5])
    ax.set_xticklabels(['NS', 'SNS'], fontsize=120)
    ax.set_yticks(np.arange(-0.5, -5.5, -1))
    ax.set_yticklabels(np.arange(1, 6), fontsize=120)
    ax.set_title(dataset, fontweight='bold', fontsize=150)

    if (i % n_columns == 0): ax.set_ylabel('Position', fontsize=20 * size)
    if (i >= n_columns * (n_rows - 1)): ax.set_xlabel('Metric', fontsize=20 * size)
    
plt.tight_layout()

In [ ]:
fig.savefig('figs/Q3_for_stress(2x8)_Oranges.pdf', format='pdf')

### Statistics

In [22]:
iso2nd = (df['NS', 'ISO'] == 2).mean()
print(f"ISO was 2nd in NS {iso2nd * 100:.2f}% of the time")
lle2nd = (df['NS', 'LLE'] == 2).mean()
print(f"LLE was 2nd in NS {lle2nd * 100:.2f}% of the time")
tsne2nd = (df['SNS', 'T-SNE'] == 2).mean()
print(f"t-SNE was 2nd in SNS {tsne2nd * 100:.2f}% of the time")

ISO was 2nd in NS 33.33% of the time
LLE was 2nd in NS 40.00% of the time
t-SNE was 2nd in SNS 80.00% of the time


In [24]:
tsne_last = (df['NS', 'T-SNE'] == 5).mean()
print(f"t-SNE was last in NS {tsne_last * 100:.2f}% of the time")
lle_last = (df['SNS', 'LLE'] == 5).mean()
print(f"LLE was last in SNS {lle_last * 100:.2f}% of the time")


t-SNE was last in NS 80.00% of the time
LLE was last in SNS 73.33% of the time
